# GAN cải tiến chuyển ảnh grayscale sang RGB
---
Notebook chuyển từ file Python, bao gồm quá trình xử lý ảnh, xây dựng Generator/Discriminator, huấn luyện và đánh giá.

In [1]:
# Cài đặt nếu chưa có (bỏ comment nếu cần)

!pip install pytorch-fid lpips



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nv

In [2]:
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import os
import numpy as np
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from pytorch_fid import fid_score
import lpips
import matplotlib.pyplot as plt
from skimage.color import lab2rgb, rgb2lab # <--- DÒNG CODE CẦN THÊM HOẶC KIỂM TRA


In [ ]:
import zipfile
import os
# giải nén file checkpoint đã lưu từ local lên 
zip_path = '/kaggle/input/checkpoints1/checkpoints1.zip'
extract_path = '/kaggle/working/checkpoints1'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

image_size = 256
batch_size = 16
lr = 0.0001
epochs = 100
SEED_SIZE = 1

INPUT_IMAGE_CHANNELS = 1
GEN_IMAGE_CHANNELS = 3

save_dir = '/kaggle/working/output2'
os.makedirs(save_dir, exist_ok=True)
save_bad = '/kaggle/working/bad_images'
os.makedirs(save_bad, exist_ok=True)
img_dir = '/kaggle/input/dataset/data1'



checkpoint_dir = '/kaggle/working/checkpoints1'
os.makedirs(checkpoint_dir, exist_ok=True)
start_epoch = 0  # để khởi động lại từ epoch cụ thể nếu resume
best_generator_loss = float('inf')



Using device: cuda


In [5]:
import shutil  # tải lại file checkpoint từ local lên input kaggle, roi chuyển sang working để load_checkpoint
input_checkpoint_path = "/kaggle/input/checkpoint11/pytorch/default/1/checkpoint_latest.pth"
target_checkpoint_path = os.path.join(checkpoint_dir, "checkpoint_latest.pth")
shutil.copy("/kaggle/input/checkpoint11/pytorch/default/1/checkpoint_latest.pth", "/kaggle/working/checkpoints1/checkpoint_latest.pth")

'/kaggle/working/checkpoints1/checkpoint_latest.pth'

In [6]:


def save_checkpoint(epoch, generator, discriminator, opt_G, opt_D, best_generator_loss, best_score):
    checkpoint = {
        'epoch': epoch,
        'generator_state_dict': generator.state_dict(),
        'discriminator_state_dict': discriminator.state_dict(),
        'opt_G_state_dict': opt_G.state_dict(),
        'opt_D_state_dict': opt_D.state_dict(),
        'best_generator_loss': best_generator_loss,
        'best_score': best_score  # Thêm dòng này
    }
    torch.save(checkpoint, os.path.join(checkpoint_dir, "checkpoint_latest.pth"))
    print(f"✅ Đã lưu checkpoint tại epoch {epoch + 1}")

def load_checkpoint(generator, discriminator, opt_G, opt_D):
    
    path = os.path.join(checkpoint_dir, "checkpoint_latest.pth")
    
    print("🔍 Đang kiểm tra checkpoint:", path)
    if os.path.exists(path):
        checkpoint = torch.load(path, weights_only=False)
        generator.load_state_dict(checkpoint['generator_state_dict'])
        discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
        opt_G.load_state_dict(checkpoint['opt_G_state_dict'])
        opt_D.load_state_dict(checkpoint['opt_D_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_generator_loss = checkpoint['best_generator_loss']
        best_score = checkpoint.get('best_score', -float('inf'))  # Thêm dòng này
        print(f"✅ Đã tải checkpoint. Tiếp tục từ epoch {start_epoch}")
        return start_epoch, best_generator_loss, best_score
    else:
        print("⚠️ Không tìm thấy checkpoint. Bắt đầu từ đầu.")
        return 0, float('inf'), -float('inf')  # Thêm return mặc định

In [7]:
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif', '.gif')
error_count = 0

for root, dirs, files in os.walk(img_dir):
    for file in files:
        if file.lower().endswith(valid_extensions):
            path = os.path.join(root, file)
            try:
                with Image.open(path) as img:
                    img.verify()
            except Exception as e:
                error_count += 1
                print(f"❌ Ảnh lỗi: {path} ({e})")
                shutil.move(path, os.path.join(save_bad, file))

print(f"✅ Đã xử lý xong: {error_count} ảnh lỗi đã được di chuyển vào {save_bad}")
class ColorizationDataset(Dataset):
    def __init__(self, root_dir, img_size=256, is_train=True):   # 256 thành 128
        self.root_dir = root_dir
        self.img_size = img_size
        self.is_train = is_train
        self.image_paths = []

        # Thu thập tất cả các đường dẫn ảnh trong thư mục gốc và các thư mục con
        for root, _, files in os.walk(root_dir):
            for file in files:
                if file.lower().endswith(valid_extensions): # Sử dụng valid_extensions đã định nghĩa
                    self.image_paths.append(os.path.join(root, file))

        # Định nghĩa các phép biến đổi ảnh
        if self.is_train:
            self.transforms = transforms.Compose([
                transforms.Resize(self.img_size + 30), # Resize lớn hơn một chút  # chỉnh 30 về 20 của 128
                transforms.RandomCrop(self.img_size),  # Cắt ngẫu nhiên
                transforms.RandomHorizontalFlip(),      # Lật ngang 
                transforms.RandomRotation(10),          # Xoay ngẫu nhiên   từ 10 về 5   của 128
                transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05), # Cải thiện ColorJitter
                transforms.ToTensor(),                  # Chuyển đổi sang Tensor (RGB, [0, 1])
                
            ])
        else: # Đối với validation/test, chỉ cần resize và chuẩn hóa
            self.transforms = transforms.Compose([
                transforms.Resize((self.img_size, self.img_size)),
                transforms.ToTensor(),
               
            ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            # Mở ảnh và chuyển sang RGB
            img_rgb_pil = Image.open(img_path).convert('RGB')
            
            # Áp dụng transforms cho ảnh RGB (PIL Image)
            img_rgb_tensor = self.transforms(img_rgb_pil) 
            
            # Chuyển đổi từ RGB Tensor về numpy array để sang L*a*b*
            # skimage.rgb2lab mong đợi numpy array với giá trị trong khoảng [0, 255]
            img_rgb_np = img_rgb_tensor.permute(1, 2, 0).numpy() # (C, H, W) -> (H, W, C)
            #img_rgb_np = (img_rgb_np * 0.5 + 0.5) * 255 # Đưa về khoảng [0, 255] nhân hai lần 255 khiến ảnh sáng, làm học sai
             # Chuẩn hóa đúng cách từ [0, 1] về [0, 255] và ép kiểu
            img_rgb_np = (img_rgb_np * 255).astype(np.uint8) # <-- Dòng đã sửa
            
            # Chuyển đổi sang không gian màu L*a*b*
            img_lab = rgb2lab(img_rgb_np).astype("float32") 
            
            # Tách kênh L và kênh a, b, sau đó chuẩn hóa về [-1, 1]
            L = img_lab[:, :, 0] / 50.0 - 1.0 # Kênh L: [0, 100] -> [-1, 1]
            ab = img_lab[:, :, 1:] / 128.0   # Kênh a, b: [-128, 128] -> [-1, 1]
            
            # Chuyển numpy array về Tensor và định dạng đúng (C, H, W)
            L = torch.from_numpy(L).unsqueeze(0) # (1, H, W)
            ab = torch.from_numpy(ab).permute(2, 0, 1) # (2, H, W)

            return L, ab # Trả về kênh L (grayscale) và kênh ab (color)

        except Exception as e:
            print(f"❌ Bỏ qua ảnh lỗi khi load (tại getitem): {img_path} ({e}). Trả về dummy tensors.")
            # Trả về dummy tensors với kích thước chính xác cho IMG_SIZE
            dummy_L = torch.zeros((1, self.img_size, self.img_size))
            dummy_ab = torch.zeros((2, self.img_size, self.img_size))
            return dummy_L, dummy_ab



dataset = ColorizationDataset(
    root_dir=img_dir,
    img_size=image_size,
    is_train=True # is_train chỉ ảnh hưởng đến augmentations, không ảnh hưởng đến việc chia dataset
)
train_dataset, test_dataset = random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Train size:", len(train_dataset), "Test size:", len(test_dataset)) 

✅ Đã xử lý xong: 0 ảnh lỗi đã được di chuyển vào /kaggle/working/bad_images
Train size: 1835 Test size: 459


In [8]:
sample_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

for L, ab in sample_loader:
    print("L shape:", L.shape)
    print("ab shape:", ab.shape)
    break


L shape: torch.Size([16, 1, 256, 256])
ab shape: torch.Size([16, 2, 256, 256])


In [9]:
# ----------- Self-Attention Block (Đã sửa đổi) -----------
class SelfAttention(nn.Module):
    """
    Block Self-Attention (Đã sửa lỗi CUDA assert).
    Mặc dù phiên bản cũ không sai về logic, nhưng đôi khi
    cách sắp xếp tensor có thể gây ra lỗi không mong muốn trên GPU.
    Phiên bản này sử dụng một cách viết phổ biến và ổn định hơn.
    """
    def __init__(self, in_dim):
        super(SelfAttention, self).__init__()
        self.query = nn.Conv2d(in_dim, max(in_dim // 8, 1), 1)
        self.key = nn.Conv2d(in_dim, max(in_dim // 8, 1), 1)
        self.value = nn.Conv2d(in_dim, in_dim, 1)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        B, C, H, W = x.size()
        # Chuyển đổi tensor để tính toán
        proj_query = self.query(x).reshape(B, -1, H * W)  # B x (C/8) x N
        proj_key = self.key(x).reshape(B, -1, H * W)      # B x (C/8) x N
        proj_value = self.value(x).reshape(B, -1, H * W)  # B x C x N

        # Tính attention map
        # (B x N x C/8) x (B x C/8 x N) -> (B x N x N)
        attention = torch.bmm(proj_key.permute(0, 2, 1), proj_query)
        attention = F.softmax(attention, dim=-1)

        # Áp dụng attention map lên value
        # (B x C x N) x (B x N x N) -> (B x C x N)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        
        # Reshape lại về kích thước ban đầu
        out = out.view(B, C, H, W)

        # Thêm kết quả vào đầu vào ban đầu
        return self.gamma * out + x

# ----------- Residual Block -----------
class ResidualBlock(nn.Module):
    """
    Block dư cơ bản để thêm vào giữa các lớp.
    """
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, in_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(in_channels)
        )

    def forward(self, x):
        return x + self.block(x)

# ----------- SE Block (Channel Attention) -----------
class SEBlock(nn.Module):
    """
    Squeeze-and-Excitation Block để tạo attention theo kênh.
    """
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.fc = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, max(in_channels // reduction, 1), 1),
            nn.ReLU(),
            nn.Conv2d(max(in_channels // reduction, 1), in_channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        w = self.fc(x)
        return x * w

# ----------- Downsampling Block -----------
def down_block(in_channels, out_channels, use_attn=False):
    """
    Hàm tạo Downsampling Block với các lớp tiêu chuẩn và SEBlock.
    """
    layers = [
        nn.Conv2d(in_channels, out_channels, 4, 2, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2, inplace=True),
        SEBlock(out_channels)
    ]
    if use_attn:
        layers.append(SelfAttention(out_channels))
    return nn.Sequential(*layers)

# ----------- Upsampling Block -----------
def up_block(in_channels, out_channels, dropout=False, use_attn=False):
    """
    Hàm tạo Upsampling Block với các lớp tiêu chuẩn và SEBlock.
    """
    layers = [
        nn.ConvTranspose2d(in_channels, out_channels, 4, 2, 1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
        SEBlock(out_channels)
    ]
    if dropout:
        layers.append(nn.Dropout(0.5))
    if use_attn:
        layers.append(SelfAttention(out_channels))
    return nn.Sequential(*layers)

# ----------- Generator Main Model -----------
class UNetGenerator(nn.Module):
    """
    Mô hình Generator dạng U-Net cải tiến để chuyển ảnh xám sang màu.
    Đầu vào: 1 kênh (L*)
    Đầu ra: 2 kênh (a* và b*)
    """
    def __init__(self, input_channels=1, output_channels=2):
        super(UNetGenerator, self).__init__()
        # Downsampling
        self.d1 = down_block(input_channels, 64)
        self.d2 = down_block(64, 128)
        self.d3 = down_block(128, 256)
        self.d4 = down_block(256, 512)

        # Middle
        self.middle = nn.Sequential(
            ResidualBlock(512),
            SelfAttention(512),
            ResidualBlock(512)
        )

        # Upsampling
        self.u1 = up_block(512 * 2, 256, dropout=True)
        self.u2 = up_block(256 * 2, 128, use_attn=True)
        self.u3 = up_block(128 * 2, 64)
        self.u4 = up_block(64 * 2, 32)

        # Final conv + thêm 1 residual block trước Tanh
        self.final = nn.Sequential(
            ResidualBlock(32),
            nn.Conv2d(32, output_channels, 3, 1, 1, bias=True),
            nn.Tanh()
        )

    def forward(self, x):
        d1 = self.d1(x)
        d2 = self.d2(d1)
        d3 = self.d3(d2)
        d4 = self.d4(d3)

        m = self.middle(d4)

        u1 = self.u1(torch.cat([m, d4], dim=1))
        u2 = self.u2(torch.cat([u1, d3], dim=1))
        u3 = self.u3(torch.cat([u2, d2], dim=1))
        u4 = self.u4(torch.cat([u3, d1], dim=1))

        out = self.final(u4)

        return out


In [10]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(Discriminator, self).__init__()

        def conv_block(in_c, out_c, normalize=True):
            layers = [nn.Conv2d(in_c, out_c, 4, 2, 1, bias=False)]
            if normalize:
                layers.append(nn.BatchNorm2d(out_c))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return nn.Sequential(*layers)

        self.block1 = conv_block(in_channels, 64, normalize=False)
        self.block2 = conv_block(64, 128)
        self.block3 = conv_block(128, 256)
        self.block4 = conv_block(256, 512)
        
        self.res = ResidualBlock(512)
        # Sử dụng SelfAttention đã sửa lỗi ở trên
        self.attn = SelfAttention(512)
        
        self.output_layer = nn.Sequential(
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0,bias=True),
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.res(x)
        x = self.attn(x)
        x = self.output_layer(x)
        return x


In [11]:
import warnings
# Tùy chọn: ẩn cảnh báo "negative Z values" từ skimage
warnings.filterwarnings("ignore", message="Conversion from CIE-LAB.*", category=UserWarning)

import numpy as np
import torch
from skimage.color import lab2rgb

def lab_to_rgb_tensor(L_channel, ab_channel):
    """
    Chuyển đổi từ tensor LAB sang tensor RGB.
    
    Args:
        L_channel (torch.Tensor): Kênh L (lightness), giá trị trong khoảng [-1, 1]. Shape: (B, 1, H, W)
        ab_channel (torch.Tensor): Kênh a và b, giá trị trong khoảng [-1, 1]. Shape: (B, 2, H, W)
        device: 'cuda' hoặc 'cpu'

    Returns:
        torch.Tensor: Tensor RGB, giá trị trong khoảng [-1, 1]. Shape: (B, 3, H, W)
    """
    device = L_channel.device  # Lấy device từ input
    # 1. Chuyển đổi tensor về numpy
    L_np = L_channel.detach().cpu().numpy()
    ab_np = ab_channel.detach().cpu().numpy()
    
    # 2. Chuẩn hóa ngược về các giá trị LAB gốc
    # L trong không gian LAB chuẩn có giá trị từ [0, 100]
    # a, b trong không gian LAB chuẩn có giá trị từ [-128, 128]
    L_np = (L_np + 1.0) / 2.0 * 100.0  # Chuẩn hóa ngược kênh L từ [-1, 1] về [0, 100]
    ab_np = ab_np * 128.0 # Chuẩn hóa ngược kênh ab từ [-1, 1] về [-128, 128]
    
    # 3. Kết hợp các kênh
    lab_np = np.concatenate((L_np, ab_np), axis=1) # (B, 3, H, W)
    lab_np = lab_np.transpose((0, 2, 3, 1)) # (B, H, W, 3)

    rgb_imgs = []
    for img_lab in lab_np:
        # img_lab có shape (H, W, 3)
        img_rgb = lab2rgb(img_lab)  # trả về ảnh RGB có dải giá trị [0, 1]
        rgb_imgs.append(img_rgb)
    
    # 4. Chuyển lại về tensor và chuẩn hóa
    rgb_imgs = np.stack(rgb_imgs, axis=0) # (B, H, W, 3)
    rgb_imgs = torch.from_numpy(rgb_imgs).permute(0, 3, 1, 2) # (B, 3, H, W)
    
    # Chuẩn hóa RGB từ [0, 1] về [-1, 1] để phù hợp với mô hình
    rgb_imgs = rgb_imgs * 2.0 - 1.0

    return rgb_imgs.to(device)

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [ ]:
import torch
torch.cuda.empty_cache()

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import lpips
import json
import cv2
import torch.optim.lr_scheduler as lr_scheduler
from skimage.metrics import peak_signal_noise_ratio as psnr_fn
from skimage.metrics import structural_similarity as ssim_fn
from pytorch_fid import fid_score
from tqdm import tqdm
import os
import zipfile
import tempfile
from torchvision import transforms
import numpy as np

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import warnings
warnings.filterwarnings("ignore", message="Conversion from CIE-LAB.*", category=UserWarning)

def zip_folder(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                filepath = os.path.join(root, file)
                arcname = os.path.relpath(filepath, folder_path)
                zipf.write(filepath, arcname)

def tensor_to_numpy(img):
    img = img.detach().cpu().clamp(0, 1)
    return img.permute(1, 2, 0).numpy()

# Giả định các hàm sau đã được định nghĩa:
# - UNetGenerator: Mô hình Generator với input_channels=1, output_channels=2
# - Discriminator: Mô hình Discriminator với in_channels=3
# - lab_to_rgb_tensor: Hàm chuyển đổi từ L*a*b* sang RGB
# - save_image: Hàm lưu ảnh từ tensor
# - save_checkpoint, load_checkpoint: Hàm lưu/tải checkpoint
# - train_loader, test_loader: DataLoader cho dữ liệu huấn luyện/kiểm tra
# - save_dir, checkpoint_dir: Thư mục lưu kết quả/checkpoint
# - device: Thiết bị (CPU/GPU)

# Khởi tạo mô hình
best_generator_weights = None
generator = UNetGenerator(input_channels=1, output_channels=2).to(device)
discriminator = Discriminator(in_channels=3).to(device)
lpips_loss_fn = lpips.LPIPS(net='alex').to(device)
l1_loss_fn = nn.L1Loss()
bce_loss_fn = nn.BCEWithLogitsLoss()

# Tối ưu hóa
opt_G = optim.Adam(generator.parameters(), lr=0.0001, betas=(0.5, 0.999))
opt_D = optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.5, 0.999))

best_score = -float('inf')
resume_training = True

if resume_training:
    start_epoch, best_generator_loss, best_score = load_checkpoint(generator, discriminator, opt_G, opt_D)
else:
    print("🚀 Đang khởi động training từ đầu...")
    start_epoch = 0
    best_generator_loss = float('inf')

# Ghi log ra file
log_file_path = os.path.join(save_dir, "training_log.txt")
log_file = open(log_file_path, "a" if resume_training else "w")
G_losses, D_losses = [], []
PSNR_scores, SSIM_scores, LPIPS_scores, FID_scores = [], [], [], []
best_epoch = -1

# Trọng số loss
LAMBDA_L1 = 200
LAMBDA_LPIPS = 20
LAMBDA_ADV = 1  # Tăng để ảnh tự nhiên hơn

test_L, test_ab_fake = None, None
epochs = 100 - start_epoch

for epoch in range(start_epoch, start_epoch + epochs):
    g_loss_epoch, d_loss_epoch = 0, 0
    psnr_total, ssim_total, lpips_total = 0, 0, 0
    n_samples = 0
    loop = tqdm(train_loader, desc=f"Epoch [{epoch + 1}/{epochs}]")

    for L_channel_real, ab_channel_real in loop:
        L_channel_real, ab_channel_real = L_channel_real.to(device), ab_channel_real.to(device)
        
        # Kiểm tra định dạng tensor
        assert L_channel_real.size(1) == 1, f"Expected 1 channel for L, got {L_channel_real.size(1)}"
        assert ab_channel_real.size(1) == 2, f"Expected 2 channels for ab, got {ab_channel_real.size(1)}"

        # TRAIN DISCRIMINATOR
        opt_D.zero_grad()
        real_img_for_D = torch.cat([L_channel_real, ab_channel_real], dim=1)
        d_real = discriminator(real_img_for_D)
        d_loss_real = bce_loss_fn(d_real, torch.ones_like(d_real) * 0.9)
        
        fake_ab_channel = generator(L_channel_real).detach()
        fake_img_for_D = torch.cat([L_channel_real, fake_ab_channel], dim=1)
        d_fake = discriminator(fake_img_for_D)
        d_loss_fake = bce_loss_fn(d_fake, torch.zeros_like(d_fake) * 0.1)
        
        d_loss = (d_loss_real + d_loss_fake) / 2
        d_loss.backward()
        opt_D.step()

        # TRAIN GENERATOR
        opt_G.zero_grad()
        fake_ab_channel = generator(L_channel_real)
        fake_img_for_D_G = torch.cat([L_channel_real, fake_ab_channel], dim=1)
        d_fake_for_G = discriminator(fake_img_for_D_G)
        adv_loss = bce_loss_fn(d_fake_for_G, torch.ones_like(d_fake_for_G) * 0.9)
        
        reconstruction_loss = l1_loss_fn(fake_ab_channel, ab_channel_real)
        
        real_rgb_for_lpips = lab_to_rgb_tensor(L_channel_real, ab_channel_real).float()
        fake_rgb_for_lpips = lab_to_rgb_tensor(L_channel_real, fake_ab_channel).float()
        perceptual_loss = lpips_loss_fn(fake_rgb_for_lpips, real_rgb_for_lpips).mean()
        
        g_loss = LAMBDA_ADV * adv_loss + LAMBDA_L1 * reconstruction_loss + LAMBDA_LPIPS * perceptual_loss 
        g_loss.backward()
        opt_G.step()

        # Tính chỉ số hình ảnh
        for i in range(real_rgb_for_lpips.size(0)):
            gt_rgb = lab_to_rgb_tensor(L_channel_real[i:i+1], ab_channel_real[i:i+1]).squeeze(0)
            pred_rgb = lab_to_rgb_tensor(L_channel_real[i:i+1], fake_ab_channel[i:i+1]).squeeze(0)
            gt_np = (gt_rgb.detach().cpu().permute(1, 2, 0).numpy() + 1) / 2
            pred_np = (pred_rgb.detach().cpu().permute(1, 2, 0).numpy() + 1) / 2
            
            psnr_total += psnr_fn(gt_np, pred_np, data_range=1)
            ssim_total += ssim_fn(gt_np, pred_np, data_range=1, channel_axis=-1)
        
        lpips_total += perceptual_loss.item() * real_rgb_for_lpips.size(0)
        n_samples += real_rgb_for_lpips.size(0)

        loop.set_postfix(G_Loss=g_loss.item(), D_Loss=d_loss.item())
        g_loss_epoch += g_loss.item()
        d_loss_epoch += d_loss.item()

    # Tính FID
    
    temp_real_dir = tempfile.mkdtemp()
    temp_gen_dir = tempfile.mkdtemp()

    test_L = L_channel_real[:128].detach()
    test_ab_fake = fake_ab_channel[:128].detach()

    for i in range(test_L.size(0)):
        real_rgb = lab_to_rgb_tensor(test_L[i:i+1], ab_channel_real[i:i+1]).squeeze(0)
        fake_rgb = lab_to_rgb_tensor(test_L[i:i+1], test_ab_fake[i:i+1]).squeeze(0)
    
        real_rgb = (real_rgb + 1) / 2
        fake_rgb = (fake_rgb + 1) / 2
    
        transforms.ToPILImage()(real_rgb).save(f"{temp_real_dir}/{i}.png")
        transforms.ToPILImage()(fake_rgb).save(f"{temp_gen_dir}/{i}.png")
    
    fid = fid_score.calculate_fid_given_paths(
        [temp_real_dir, temp_gen_dir],
        batch_size=16,
        device=device,
        dims=2048
    )




    # Cập nhật metrics
    G_avg = g_loss_epoch / len(train_loader)
    D_avg = d_loss_epoch / len(train_loader)
    psnr_avg = psnr_total / n_samples
    ssim_avg = ssim_total / n_samples
    lpips_avg = lpips_total / n_samples
    FID_scores.append(fid)

    G_losses.append(G_avg)
    D_losses.append(D_avg)
    PSNR_scores.append(psnr_avg)
    SSIM_scores.append(ssim_avg)
    LPIPS_scores.append(lpips_avg)

    # Ghi metrics ra file JSON
    metrics = {
        'G_losses': G_losses,
        'D_losses': D_losses,
        'PSNR_scores': PSNR_scores,
        'SSIM_scores': SSIM_scores,
        'LPIPS_scores': LPIPS_scores,
        'FID_scores': FID_scores
    }
    with open(os.path.join(save_dir, 'metrics.json'), 'w') as f:
        json.dump(metrics, f)

    # Ghi log
    log_file.write(
        f"Epoch {epoch + 1:03d}:\n"
        f"  G_Loss:  {G_avg:.4f}\n"
        f"  D_Loss:  {D_avg:.4f}\n"
        f"  PSNR:    {psnr_avg:.2f}\n"
        f"  SSIM:    {ssim_avg:.4f}\n"
        f"  LPIPS:   {lpips_avg:.4f}\n"
        f"  FID:     {fid:.4f}\n"
        "-------------------------\n"
    )
    log_file.flush()

    # Lưu ảnh kết quả
    fake_rgb_to_save = lab_to_rgb_tensor(test_L, test_ab_fake)
    ##print("Fake RGB tensor stats:", fake_rgb_to_save.min().item(), fake_rgb_to_save.max().item())
    # Chuyển từ [-1, 1] về [0, 1]  để hiển thị ảnh vì hàm lab_to_rgb_tensor đang ở dải [-1, 1]
    fake_rgb_to_save = (fake_rgb_to_save + 1.0) / 2.0 
    save_image(
    (fake_rgb_to_save[:30]),
    os.path.join(save_dir, f"epoch_{epoch + 1}.png"),
    nrow=5,
    normalize=False  # hoặc bỏ normalize luôn
)


    # Lưu mô hình tốt nhất
    score = -10 * lpips_avg + 5.0 * ssim_avg +  psnr_avg # kh cần fid lắm vì ưu tiên 3 cái kia hơn - 0.02 * fid
    
    

    if score > best_score:
        best_score = score
        best_generator_weights = generator.state_dict()
        best_epoch = epoch + 1
        torch.save(best_generator_weights, os.path.join(save_dir, 'generator_best.pth'))
        print(f"✅ Đã lưu mô hình tốt nhất tại epoch {best_epoch} với PSNR = {psnr_avg:.4f}, LPIPS = {lpips_avg:.4f}, SSIM = {ssim_avg:.4f}, FID = {fid:.4f}, Score = {best_score:.4f}")
   
    save_checkpoint(epoch, generator, discriminator, opt_G, opt_D, best_generator_loss, best_score)

    if (epoch + 1) % 10 == 0:
        print(f"🗜️ Đang nén output tại epoch {epoch + 1}...")
        zip_folder(save_dir, f'/kaggle/working/output_epoch{epoch + 1}.zip')

    if (epoch + 1) % 20 == 0:
        print(f"🗜️ Đang nén checkpoint tại epoch {epoch + 1}...")
        zip_folder(checkpoint_dir, f'/kaggle/working/checkpoints_epoch{epoch + 1}.zip')
    
# Lưu mô hình cuối cùng
torch.save(generator.state_dict(), os.path.join(save_dir, 'generator_final.pth'))
torch.save(discriminator.state_dict(), os.path.join(save_dir, 'discriminator_final.pth'))

log_file.close()

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 206MB/s]  


Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth
🔍 Đang kiểm tra checkpoint: /kaggle/working/checkpoints1/checkpoint_latest.pth
✅ Đã tải checkpoint. Tiếp tục từ epoch 52


Epoch [53/48]: 100%|██████████| 115/115 [04:09<00:00,  2.17s/it, D_Loss=0.486, G_Loss=20]  
Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:00<00:00, 266MB/s]


100%|██████████| 1/1 [00:00<00:00,  3.30it/s]


...

✅ Đã lưu checkpoint tại epoch 99


Epoch [100/48]: 100%|██████████| 115/115 [04:06<00:00,  2.14s/it, D_Loss=0.38, G_Loss=25]   


100%|██████████| 1/1 [00:00<00:00,  5.11it/s]


100%|██████████| 1/1 [00:00<00:00,  5.24it/s]


✅ Đã lưu checkpoint tại epoch 100
🗜️ Đang nén output tại epoch 100...
🗜️ Đang nén checkpoint tại epoch 100...


In [13]:
import zipfile
import torch.nn.functional as F
def evaluate_model(model, dataloader, real_dir, gen_dir):
    os.makedirs(real_dir, exist_ok=True)
    os.makedirs(gen_dir, exist_ok=True)

    lpips_fn = lpips.LPIPS(net='alex').to(device)
    to_pil = transforms.ToPILImage()

    def denormalize(tensor):
        return tensor * 0.5 + 0.5

    psnr_list, ssim_list, lpips_list = [], [], []

    model.eval()
    with torch.no_grad():
        for i, (L_channel, ab_channel) in enumerate(tqdm(dataloader)):
            L_channel, ab_channel = L_channel.to(device), ab_channel.to(device)
            fake_ab = model(L_channel)  # Đảm bảo L_channel là 1 kênh

            for j in range(L_channel.size(0)):
                real_rgb = lab_to_rgb_tensor(L_channel[j:j+1], ab_channel[j:j+1]).squeeze(0)
                fake_rgb = lab_to_rgb_tensor(L_channel[j:j+1], fake_ab[j:j+1]).squeeze(0)

                real_np = denormalize(real_rgb).cpu().permute(1, 2, 0).numpy()
                fake_np = denormalize(fake_rgb).cpu().permute(1, 2, 0).numpy()

                psnr_list.append(psnr_fn(real_np, fake_np, data_range=1))
                ssim_list.append(ssim_fn(real_np, fake_np, data_range=1, channel_axis=-1))
                lpips_val = lpips_fn(real_rgb.unsqueeze(0).to(device), fake_rgb.unsqueeze(0).to(device)).item()
                lpips_list.append(lpips_val)

                to_pil(denormalize(real_rgb)).save(f"{real_dir}/real_{i}_{j}.png")
                to_pil(denormalize(fake_rgb)).save(f"{gen_dir}/gen_{i}_{j}.png")

    fid = fid_score.calculate_fid_given_paths([real_dir, gen_dir], batch_size=50, device=device, dims=2048)

    psnr_mean = np.mean(psnr_list)
    ssim_mean = np.mean(ssim_list)
    lpips_mean = np.mean(lpips_list)

    metrics_path = "/kaggle/working/metrics_result.txt"
    with open(metrics_path, "w") as f:
        f.write(f"PSNR: {psnr_mean:.4f}\n")
        f.write(f"SSIM: {ssim_mean:.4f}\n")
        f.write(f"LPIPS: {lpips_mean:.4f}\n")
        f.write(f"FID: {fid:.4f}\n")

    with zipfile.ZipFile("/kaggle/working/metrics_result.zip", "w") as zipf:
        zipf.write(metrics_path, arcname="metrics_result.txt")

    for dir_path, zip_path in [(real_dir, "/kaggle/working/real_images2.zip"), (gen_dir, "/kaggle/working/generated_images2.zip")]:
        with zipfile.ZipFile(zip_path, "w") as zipf:
            for root, _, files in os.walk(dir_path):
                for file in files:
                    full_path = os.path.join(root, file)
                    arcname = os.path.relpath(full_path, dir_path)
                    zipf.write(full_path, arcname=arcname)

    return {
        "PSNR": psnr_mean,
        "SSIM": ssim_mean,
        "LPIPS": lpips_mean,
        "FID": fid,
        "psnr_list": psnr_list,
        "ssim_list": ssim_list,
        "lpips_list": lpips_list
    }
real_dir = '/kaggle/working/real_images2'
gen_dir = '/kaggle/working/generated_images2'
generator = UNetGenerator().to(device)
generator.load_state_dict(torch.load(os.path.join(save_dir, 'generator_best.pth')))
#generator.load_state_dict(torch.load("/kaggle/input/genarator_bestv2/pytorch/default/1/generator_best.pth"))  # load lại mô hình tốt nhất từ
#local để tính giá trị

generator.eval()
metrics = evaluate_model(generator, test_loader, real_dir, gen_dir)
print(metrics)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth


100%|██████████| 10/10 [00:02<00:00,  3.94it/s]


{'PSNR': 21.08641275354926, 'SSIM': 0.88055086, 'LPIPS': 0.23457356521247, 'FID': 78.94952966935972, 'psnr_list': [20.13381149373286, 19.71510197245642, 24.20267721856691, 21.600782037907102, 16.47741405993144, 17.4081097702538, 26.50317090684908, 22.501839684947143, 24.008022063116982, 22.0026635873589, 27.78388645665774, 20.494851996560136, 18.23755843845555, 22.3554058476835, 21.245644774708264, 25.298298193071503, 14.174237046393143, 17.650933166808592, 20.818670709181482, 26.367184127161572, 24.22363766105263, 18.106956949601024, 24.70736442460654, 20.50474596653696, 24.07826857091861, 22.256121412854103, 23.493402110122435, 18.35375920062553, 17.999532060697536, 25.316722872990894, 20.363848445589184, 25.542234906119823, 22.247497123424395, 20.142973310068957, 25.29665993714136, 28.622840746882595, 23.56769647391676, 25.279033471341425, 24.18230969499705, 15.558484543458974, 24.863526090031467, 23.804763573341155, 23.52118832510017, 15.467398043358996, 26.065748221913125, 27.2873

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Đọc file metrics_result.txt từ input
file_path = "/kaggle/input/training/pytorch/default/1/trainning_chy.txt"

df = pd.read_csv(file_path)

# Trích xuất các cột
epochs = df['Epoch']
G_losses = df['G_loss']
D_losses = df['D_loss']
PSNR = df['PSNR']
SSIM = df['SSIM']
LPIPS = df['LPIPS']

# Vẽ biểu đồ
plt.figure(figsize=(14, 10))

plt.subplot(2, 2, 1)
plt.plot(epochs, G_losses, label='G Loss')
plt.plot(epochs, D_losses, label='D Loss')
plt.title('Loss qua các Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(epochs, PSNR, color='green')
plt.title('PSNR theo Epoch')
plt.xlabel('Epoch')
plt.ylabel('PSNR')

plt.subplot(2, 2, 3)
plt.plot(epochs, SSIM, color='blue')
plt.title('SSIM theo Epoch')
plt.xlabel('Epoch')
plt.ylabel('SSIM')

plt.subplot(2, 2, 4)
plt.plot(epochs, LPIPS, color='red')
plt.title('LPIPS theo Epoch')
plt.xlabel('Epoch')
plt.ylabel('LPIPS')

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(range(start_epoch , start_epoch + 1 + len(G_losses)), G_losses, label="G Loss", color='blue')
plt.plot(range(start_epoch , start_epoch + 1 + len(D_losses)), D_losses, label="D Loss", color='red')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("G and D Loss over Epochs")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "loss_over_epochs.png"))
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(range(start_epoch + 1, start_epoch + 1 + len(PSNR_scores)), PSNR_scores, marker='o', color='green', label="PSNR")
plt.xlabel("Epoch")
plt.ylabel("PSNR")
plt.title("PSNR over Epochs")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "psnr_over_epochs.png"))
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(range(start_epoch + 1, start_epoch + 1 + len(SSIM_scores)), SSIM_scores, marker='o', color='orange', label="SSIM")
plt.xlabel("Epoch")
plt.ylabel("SSIM")
plt.title("SSIM over Epochs")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "ssim_over_epochs.png"))
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(range(start_epoch + 1, start_epoch + 1 + len(LPIPS_scores)), LPIPS_scores, marker='o', color='purple', label="LPIPS")
plt.xlabel("Epoch")
plt.ylabel("LPIPS")
plt.title("LPIPS over Epochs")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, "lpips_over_epochs.png"))
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(range(start_epoch + 1, start_epoch + 1 + len(FID_scores)), FID_scores, label='FID', color='brown')
plt.title('FID per Epoch')
plt.xlabel('Epoch')
plt.ylabel('FID')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(save_dir, 'fid_curve.png'))
plt.show()


In [ ]:
# Vẽ PSNR
import matplotlib.pyplot as plt

psnr_list = metrics['psnr_list']
plt.figure(figsize=(8, 5))
plt.plot(psnr_list, marker='o', color='blue', label='PSNR')
plt.title("PSNR over Test Images")
plt.xlabel("Image Index")
plt.ylabel("PSNR")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("/kaggle/working/psnr_over_images.png")

In [ ]:
ssim_list = metrics['ssim_list']
plt.figure(figsize=(8, 5))
plt.plot(ssim_list, marker='s', color='green', label='SSIM')
plt.title("SSIM over Epochs")
plt.xlabel("Image Index")
plt.ylabel("SSIM")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("/kaggle/working/ssim_over_epochs.png")
plt.show()


In [ ]:
lpips_list = metrics['lpips_list']
plt.figure(figsize=(8, 5))
plt.plot( lpips_list, marker='^', color='red', label='LPIPS')
plt.title("LPIPS over Epochs")
plt.xlabel("Image Index")
plt.ylabel("LPIPS")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("/kaggle/working/lpips_over_epochs.png")
plt.show()
